In [1]:
#do this later from admin: !python -m pip install --upgrade pip


In [1]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#API Packages: 
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

#Modeling
#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

#NLP Processing
'''from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer'''

#Time analysis
import time
import datetime
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [6]:
my_api_file = open("./mykey.txt", 'r')
my_api_str = my_api_file.read()
my_api_file.close()

In [7]:
DEVELOPER_KEY = my_api_str
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
# all views over 400M https://www.youtube.com/playlist?list=PLirAqAtl_h2r5g8xGajEwdXd3x1sZh8hC

https://www.youtube.com/results?filters=video&lclk=video&search_query=a%7Cb%7Cc%7Cd%7Ce%7Cf%7Cg%7Ch%7Ci%7Cj%7Ck%7Cl%7Cm%7Cn%7Co%7Cp%7Cq%7Cr%7Cs%7Ct%7Cu%7Cv%7Cw%7Cx%7Cy%7Cz%7C1%7C1%7C2%7C3%7C4%7C5%7C6%7C7%7C8%7C9%7C0&search_sort=video_view_count


EYYYYYYYY
https://github.com/youtube/api-samples/blob/master/python/search.py


In [9]:
# This sample executes a search request for the specified search term.

"""import argparse

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError"""

'import argparse\n\nfrom googleapiclient.discovery import build\nfrom googleapiclient.errors import HttpError'

In [3]:
#This function returns a list of videos according to search query 'q' as if you were typing it into the bar
#The out put is a json list, predominantly used to get videoId's
def youtube_search_list(q, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.search().list(
        q=q,
        part='id,snippet',
        pageToken=token,
        maxResults=max_results,
        order=sort_method
      ).execute()
    
    return search_response

In [4]:
#takes output json list from search list and returns metadata on individual videos
def youtube_search_video(q='spinners', max_results=50, sort_method='viewCount', token=None):
    
    order = "viewCount"
    q=q
    max_results = max_results
    sort_method = sort_method
    token = token
    location = None
    location_radius = None
    
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,         
    developerKey=DEVELOPER_KEY)
    
    #Return list of matching records up to max_search
    search_result = youtube_search_list(q, max_results, sort_method, token)
    next_tok = search_result['nextPageToken']
    
    videos_list = []
    for search_result in search_result.get("items", []):
        
        if search_result["id"]["kind"] == 'youtube#video':
            temp_dict_ = {}
            #Available from initial search
            temp_dict_['title'] = search_result['snippet']['title']  
            temp_dict_['vidId'] = search_result['id']['videoId']  
            
            #Secondary call to find statistics results for individual video
            response = youtube.videos().list(
                part='statistics, snippet, contentDetails', 
                id=search_result['id']['videoId']
                    ).execute()  
            response_statistics = response['items'][0]['statistics']
            response_snippet = response['items'][0]['snippet']
            response_content= response['items'][0]['contentDetails']
            
            
            snippet_list = ['publishedAt','channelId', 'description', 
                            'channelTitle', 'tags', 'categoryId', 
                            'liveBroadcastContent', 'defaultLanguage', ]
            for val in snippet_list:
                try:
                    temp_dict_[val] = response_snippet[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'    
            
            stats_list = ['favoriteCount', 'viewCount', 'likeCount', 
                          'dislikeCount', 'commentCount']
            for val in stats_list:
                try:
                    temp_dict_[val] = response_statistics[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
            
            for val in response_content.keys():
                try:
                    temp_dict_[val] = response_content[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
                    
            #add back to main list
            videos_list.append(temp_dict_)
            
    return videos_list, next_tok

In [15]:
#Instatiate datframe using first results. 
#Allows for simple forloop
videos_list, next_tok = youtube_search_video('Music Video')
df_videos = pd.DataFrame(videos_list)
df_videos['request_token'] = next_tok #for assessment of run.

count=1
while next_tok != 'last_page':

    videos_list, next_tok = youtube_search_video(q='Music Video', token=next_tok)
    print('    found page ', count)
    df_small_vids = pd.DataFrame(videos_list)
    df_small_vids['request_token'] = next_tok
    df_videos = pd.concat([df_videos, df_small_vids], sort=False)
    count += 1
    
    #create unique text file to track progress
    now_month = datetime.datetime.now().month
    now_day = datetime.datetime.now().day
    now_hour = datetime.datetime.now().hour
    now_minute = datetime.datetime.now().minute
    output_title = './data/test/iteration{}_{}.{}_{}{}.csv'.format(count, now_month, now_day, now_hour, now_minute)
    df_videos.to_csv(output_title, index=False)
    time.sleep(2)

In [17]:
df_videos['viewCount'] = df_videos['viewCount'].astype('int64')

In [18]:
df_videos['viewCount'].min()

351378

In [19]:
#it looks like youtube doesn't want to return too many videos like this
#perhaps filtering by viewcount limits the effectiveness of relevance
df_videos.shape[0]

325

In [24]:
#Given current shape, this is how many irrelevant videos I could use without 
#creating an inbalanced class
print('     max irrelevant: {}\n    final df size: {}'.format( df_videos.shape[0] / .25, df_videos.shape[0] / .25+df_videos.shape[0]))

     max irrelevant: 1300.0
    final df size: 1625.0


In [25]:
print('    num features: {}'.format(df_videos.shape[1]))

    num features: 24


### Data Strategy -- Same as Readme
I'd like to hit a dataframe row count of 10,000, with a minimum of 2k to support nlk processing. Additionally, I'm looking at expanding features by incorporating google trends results and topic analysis so this would require some more data points. To maintain a good feature / observation ratio.   

Ultimately, I'm going to be aiming for regression-based analysis to project view count. 

There are two possible paths:  
1. Stick with videos only, (see below for expansion)
2. Expand scope to all videos and redefine the problem.  
  
  
Ways to expand on data if youtube wants to stop here :
* Most viewed vevo videos of all time (estimated to add 100)
* Country, hip hop, pop, etc. hits (feels time intensive)
* Do some additional searching from billboard or spotify to get specific songs, artists
* Sourt by relevance instead (probably good for another 500)
* Branch out from artists in top scoring (would help establish non-hit basis so I'm not overfitting Psy songs, for example. probably adds 2000 records easy)
* Look at related search terms like remix, lyric video, etc, which would also expand the discussion to identify how people look for music.

  
 There may be some duplicates in there, so I will have to consider how to treat those. I'm fine with including the same song twice. 

In [27]:
'''Hold off on this one until I figure out how to tie in google trends. 
Probably we want only comments from peak viewership plus marks at half peak on either side. 
Right now, most relevant /recent comments are all stupid meme stuff, e.g. who is watching in 2018'''

#This function will grab 50 comments at a time from the video. 
def youtube_comment_list(video_id, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.commentThreads().list(
        videoId = video_id,
        part='snippet, replies',
        pageToken=token,
        order = 'relevance',
        maxResults=max_results,
      ).execute()
    
    return search_response

In [28]:
psy = youtube_comment_list('9bZkp7q19f0')

psy['items'][1]['snippet']['topLevelComment']['snippet']['textDisplay']

'Sep 24th?'

In [ ]:
#Not super informative